<a href="https://colab.research.google.com/github/sassm1t/Detection-of-Clickbait-titles-using-deep-learning/blob/main/roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Specify GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/content/merged_data.csv")

# Split dataset
train_text, temp_text, train_labels, temp_labels = train_test_split(df['headline'], df['clickbait'],
                                                                  random_state=2018,
                                                                  test_size=0.3,
                                                                  stratify=df['clickbait'])

val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
                                                                random_state=2018,
                                                                test_size=0.5,
                                                                stratify=temp_labels)

# Load RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Tokenize and encode sequences
train_encodings = tokenizer(train_text.tolist(), padding=True, truncation=True, max_length=25)
val_encodings = tokenizer(val_text.tolist(), padding=True, truncation=True, max_length=25)
test_encodings = tokenizer(test_text.tolist(), padding=True, truncation=True, max_length=25)

# Convert lists to tensors
train_seq = torch.tensor(train_encodings['input_ids'])
train_mask = torch.tensor(train_encodings['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(val_encodings['input_ids'])
val_mask = torch.tensor(val_encodings['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(test_encodings['input_ids'])
test_mask = torch.tensor(test_encodings['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

# Define batch size
batch_size = 32

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

val_dataset = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)

# Load RoBERTa model
roberta = AutoModel.from_pretrained("roberta-base")

# Freeze RoBERTa parameters
for param in roberta.parameters():
    param.requires_grad = False

# Define custom architecture on top of RoBERTa
class RoBERTa_Arch(nn.Module):
    def __init__(self, roberta):
        super(RoBERTa_Arch, self).__init__()
        self.roberta = roberta
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(roberta.config.hidden_size, 512)
        self.fc2 = nn.Linear(512, 2)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.fc1(pooled_output)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x



# Instantiate model
model = RoBERTa_Arch(roberta)

# Move model to GPU
model = model.to(device)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define training parameters
epochs = 10  # Set a maximum number of epochs
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# Initialize variables for early stopping
best_val_loss = float('inf')
best_model_state_dict = None
patience = 3  # Number of epochs to wait before early stopping
counter = 0  # Counter for early stopping

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

        loss.backward()
        optimizer.step()

    # Calculate average loss and accuracy
    avg_loss = total_loss / len(train_dataloader)
    accuracy = correct_predictions / total_predictions

    print(f'Epoch {epoch + 1}:')
    print(f'  Training Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}')

    # Validation loop
    model.eval()
    total_correct = 0
    total_samples = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    val_loss /= len(val_dataloader)
    val_accuracy = total_correct / total_samples

    print(f'  Validation Loss: {val_loss:.4f} | Accuracy: {val_accuracy:.4f}')

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state_dict = model.state_dict()
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

# Load best model state
if best_model_state_dict is not None:
    model.load_state_dict(best_model_state_dict)


# Define test data loader
test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Evaluate on the test set
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, 1)

        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

test_accuracy = total_correct / total_samples
print(f'Test Accuracy: {test_accuracy:.4f}')



Epoch 1:
  Training Loss: 0.1154 | Accuracy: 0.9589
  Validation Loss: 0.0613 | Accuracy: 0.9831
Epoch 2:
  Training Loss: 0.1102 | Accuracy: 0.9609
  Validation Loss: 0.1209 | Accuracy: 0.9590
Epoch 3:
  Training Loss: 0.1151 | Accuracy: 0.9588
  Validation Loss: 0.0726 | Accuracy: 0.9767
Epoch 4:
  Training Loss: 0.1088 | Accuracy: 0.9605
  Validation Loss: 0.0637 | Accuracy: 0.9794
Early stopping triggered.
Test Accuracy: 0.9809


In [ ]:
!pip install scikit-learn


In [ ]:
# Define test data loader
test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Evaluate on the test set
model.eval()
total_correct = 0
total_samples = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, 1)

        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

test_accuracy = total_correct / total_samples
print(f'Test Accuracy: {test_accuracy:.4f}')

# Print classification report
from sklearn.metrics import classification_report

print("Classification Report:")
print(classification_report(all_labels, all_predictions, target_names=['Non-clickbait', 'Clickbait']))


Test Accuracy: 0.9809
Classification Report:
               precision    recall  f1-score   support

Non-clickbait       0.99      0.98      0.98      3313
    Clickbait       0.98      0.99      0.98      3245

     accuracy                           0.98      6558
    macro avg       0.98      0.98      0.98      6558
 weighted avg       0.98      0.98      0.98      6558

